# Interactive Classification with KNN & SVM (Iris Dataset)

## Step 1: Install & Import Libraries

In [ ]:
# Install Plotly for interactive plots
!pip install plotly

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go

## Step 2: Load & Preprocess Data

In [ ]:
# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Step 3: Dimensionality Reduction (PCA)

In [ ]:
# Reduce to 3 principal components
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.2f}")

## Step 4: Model Training & Evaluation

### K-Nearest Neighbors

In [ ]:
# Hyperparameter tuning
param_grid = {'n_neighbors': np.arange(1, 15)}
knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
knn.fit(X_train_scaled, y_train)

# Evaluation
y_pred_knn = knn.predict(X_test_scaled)
print(f"Best K: {knn.best_params_['n_neighbors']}")
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")
print(classification_report(y_test, y_pred_knn, target_names=target_names))

### Support Vector Machine

In [ ]:
# Hyperparameter tuning
param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}
svm = GridSearchCV(SVC(), param_grid, cv=5)
svm.fit(X_train_scaled, y_train)

# Evaluation
y_pred_svm = svm.predict(X_test_scaled)
print(f"Best Params: {svm.best_params_}")
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
print(classification_report(y_test, y_pred_svm, target_names=target_names))

## Step 5: Interactive 3D Visualization

### True Labels

In [ ]:
# Create interactive plot
fig = px.scatter_3d(
    x=X_test_pca[:,0], y=X_test_pca[:,1], z=X_test_pca[:,2],
    color=y_test,
    color_continuous_scale=px.colors.sequential.Viridis,
    labels={'color': 'Species'},
    title="True Labels in 3D PCA Space"
)
fig.update_layout(
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    )
)
fig.show()

### KNN Predictions

In [ ]:
fig = go.Figure()

# Correct predictions
correct = (y_pred_knn == y_test)
fig.add_trace(go.Scatter3d(
    x=X_test_pca[correct, 0], y=X_test_pca[correct, 1], z=X_test_pca[correct, 2],
    mode='markers',
    marker=dict(size=5, color='green', opacity=0.8),
    name='Correct'
))

# Incorrect predictions
fig.add_trace(go.Scatter3d(
    x=X_test_pca[~correct, 0], y=X_test_pca[~correct, 1], z=X_test_pca[~correct, 2],
    mode='markers',
    marker=dict(size=8, color='red', symbol='x', opacity=0.8),
    name='Incorrect'
))

fig.update_layout(
    title='KNN Predictions (Green=Correct, Red=Incorrect)',
    scene=dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3')
)
fig.show()

### SVM Predictions

In [ ]:
fig = go.Figure()

correct = (y_pred_svm == y_test)
fig.add_trace(go.Scatter3d(
    x=X_test_pca[correct, 0], y=X_test_pca[correct, 1], z=X_test_pca[correct, 2],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Correct'
))

fig.add_trace(go.Scatter3d(
    x=X_test_pca[~correct, 0], y=X_test_pca[~correct, 1], z=X_test_pca[~correct, 2],
    mode='markers',
    marker=dict(size=8, color='orange', symbol='x', opacity=0.8),
    name='Incorrect'
))

fig.update_layout(
    title='SVM Predictions (Blue=Correct, Orange=Incorrect)',
    scene=dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3')
)
fig.show()

## Key Features:
1. **Interactive 3D Plots**: Rotate/zoom using mouse
2. **Color Coding**: 
   - True labels: Viridis color scale
   - Correct/incorrect predictions: Green/Red (KNN), Blue/Orange (SVM)
3. **Performance Metrics**: Accuracy + classification reports
4. **Hyperparameter Tuning**: GridSearchCV for both algorithms